0813更新：

combine-model： 把词向量和字向量信息全部丢进来训练。

一共有4路输入，wd 和 ch 各两路。

title 部分使用 bigru, content 部分使用 han-bigru.

对 title 和 content 的输出，拼起来，得到维度为 hidden_size × 2 × 4

在后边的 fc 中加上 dropout 0.5

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import pickle
import os
import sys
import shutil
import tqdm
import time


time0 = time.time()
print('Starting ...')
model_name = 'c2-1-bigru-256'                    # 模型名称
wd_embedding = np.load('../data/W_embedding.npy').astype(np.float32)                      # 导入预训练好的词向量
ch_embedding = np.load('../data/ch_W_embedding.npy').astype(np.float32)  # 字向量
model_path = '../ckpt/' + model_name + '/'                  # 模型保存位置
summary_path = '../summary/' + model_name + '/'             # summary 位置
result_path = '../result/' + model_name + '.csv'            # result.csv 位置
scores_path = '../scores/' + model_name + '.npy'            # scores.npy 位置
local_scores_path = '../local_scores/' + model_name + '.npy'


if not os.path.exists(model_path):
    os.makedirs(model_path)         
model_path = model_path + 'model.ckpt'
if os.path.exists(summary_path):   # 删除原来的 summary 文件，避免重合
    print('removed the existing summary files.')
    shutil.rmtree(summary_path)
os.makedirs(summary_path)          # 然后再次创建
    
# ##################### config ######################
n_step1 = wd_sent_len = 30                   # title句子长度
n_step2 = wd_doc_len = 10                # content 长度
n_step3 = ch_sent_len = 52
n_step4 = ch_doc_len = 10
input_size = embedding_size = 256       # 字向量长度
n_class = 1999                          # 类别总数
hidden_size = 256    # 隐含层节点数
fc_hidden_size = 1024                   # fc 层节点数
title_attention_size = sent_attention_size = doc_attention_size = hidden_size * 2
n_layer = 1        # bi-gru 层数
max_grad_norm = 1.0  # 最大梯度（超过此值的梯度将被裁剪）
global_step = 0
print('Prepared, costed time %g s.' % (time.time() - time0))

Starting ...
removed the existing summary files.
Prepared, costed time 0.502059 s.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers

'''
双端 GRU，知乎问题多标签分类。
'''
print('Building model ...')
keep_prob = tf.placeholder(tf.float32, [])
lr = tf.placeholder(tf.float32)
batch_size = tf.placeholder(tf.int32, [])  # 注意类型必须为 tf.int32
tst = tf.placeholder(tf.bool)
n_updates = tf.placeholder(tf.int32)      # training iteration,传入 bn 层
update_emas = list()                       # BN 层中所有的更新操作


def weight_variable(shape, name):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def batchnorm(Ylogits, is_test, num_updates, offset, convolutional=False):
    """batchnormalization.
    Args:
        Ylogits: 1D向量或者是3D的卷积结果。
        num_updates: 迭代的global_step
        offset：表示beta，全局均值；在 RELU 激活中一般初始化为 0.1。
        scale：表示lambda，全局方差；在 sigmoid 激活中需要，这 RELU 激活中作用不大。
        m: 表示batch均值；v:表示batch方差。
        bnepsilon：一个很小的浮点数，防止除以 0.
    Returns:
        Ybn: 和 Ylogits 的维度一样，就是经过 Batch Normalization 处理的结果。
        update_moving_everages：更新mean和variance，主要是给最后的 test 使用。
    """
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, num_updates) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    if convolutional:
        mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
    else:
        mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_everages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_everages

def gru_cell():
    with tf.variable_scope('gru_cell'):
        cell = rnn.GRUCell(hidden_size, reuse=tf.get_variable_scope().reuse)
    return rnn.DropoutWrapper(cell, output_keep_prob=keep_prob)
  
def bi_gru(inputs, seg_num=batch_size):
    """build the bi-GRU network. Return the encoder represented vector.
    n_step: 句子的词数量；或者文档的句子数。
    seg_num: 序列的数量，原本应该为 batch_size, 但是这里将 batch_size 个 doc展开成很多个句子。
    """
    cells_fw = [gru_cell() for _ in range(n_layer)]
    cells_bw = [gru_cell() for _ in range(n_layer)]
    initial_states_fw = [cell_fw.zero_state(seg_num, tf.float32) for cell_fw in cells_fw]
    initial_states_bw = [cell_bw.zero_state(seg_num, tf.float32) for cell_bw in cells_bw] 
    outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, inputs, 
                        initial_states_fw = initial_states_fw, initial_states_bw = initial_states_bw, dtype=tf.float32)
    # outputs: Output Tensor shaped: seg_num, max_time, layers_output]，其中layers_output=hidden_size * 2 在这里。
    return outputs
   
def task_specific_attention(inputs, output_size,
                            initializer=layers.xavier_initializer(),
                            activation_fn=tf.tanh, scope=None):
    """
    Performs task-specific attention reduction, using learned
    attention context vector (constant within task of interest).
    Args:
        inputs: Tensor of shape [batch_size, units, input_size]
            `input_size` must be static (known)
            `units` axis will be attended over (reduced from output)
            `batch_size` will be preserved
        output_size: Size of output's inner (feature) dimension
    Returns:
        outputs: Tensor of shape [batch_size, output_dim].
    """
    assert len(inputs.get_shape()) == 3 and inputs.get_shape()[-1].value is not None
    with tf.variable_scope(scope or 'attention') as scope:
        # u_w, attention 向量
        attention_context_vector = tf.get_variable(name='attention_context_vector',
                                                   shape=[output_size],
                                                   initializer=initializer,
                                                   dtype=tf.float32)
        # 全连接层，把 h_i 转为 u_i ， shape= [batch_size, units, input_size] -> [batch_size, units, output_size]
        input_projection = layers.fully_connected(inputs, output_size,
                                                  activation_fn=activation_fn,
                                                  scope=scope)
        # 输出 [batch_size, units]
        vector_attn = tf.reduce_sum(tf.multiply(input_projection, attention_context_vector), axis=2, keep_dims=True)
        attention_weights = tf.nn.softmax(vector_attn, dim=1)   
        tf.summary.histogram('attention_weigths', attention_weights)
        weighted_projection = tf.multiply(inputs, attention_weights)   # ???源代码,原文应该是对 hi 加权！！
        outputs = tf.reduce_sum(weighted_projection, axis=1)
        return outputs  # 输出 [batch_size, units]

def bigru_reference(inputs):
    """双端 gru 模型。title部分。
    Args: 
        X_inputs: tensor.shape=(batch_size, title_len)
    Returns:
        title_outputs: tensor.shape=(batch_size, hidden_size(*2 for bigru))
    """
    with tf.variable_scope('title_encoder'):  # 生成 title 的向量表示
        title_outputs = bi_gru(inputs)
        title_attn_outputs = task_specific_attention(title_outputs, title_attention_size)   
    return title_attn_outputs   # shape = [batch_size, hidden_size*2]


def han_reference(inputs, sent_len, doc_len=10):
    """分层 attention 模型。content部分。
    Args: 
        X_inputs: tensor.shape=(batch_size, doc_len*sent_len)
    Returns:
        doc_attn_outputs: tensor.shape=(batch_size, hidden_size(*2 for bigru))
    """
    sent_inputs = tf.reshape(inputs,[batch_size*doc_len, sent_len, embedding_size]) # shape=(?, 40, 256)
    with tf.variable_scope('sentence_encoder'):  # 生成句向量
        sent_outputs = bi_gru(sent_inputs, seg_num=batch_size*doc_len)
        sent_attn_outputs = task_specific_attention(sent_outputs, sent_attention_size) # [batch_size*doc_len, hidden_size*2]
        with tf.variable_scope('dropout'):
            sent_attn_outputs = tf.nn.dropout(sent_attn_outputs, keep_prob)
    with tf.variable_scope('doc_encoder'):  # 生成句向量
        doc_inputs = tf.reshape(sent_attn_outputs, [batch_size, doc_len, hidden_size*2])
        doc_outputs = bi_gru(doc_inputs)  # [batch_size, doc_len, hidden_size*2]
        doc_attn_outputs = task_specific_attention(doc_outputs, doc_attention_size) # [batch_size, hidden_size*2]
    return doc_attn_outputs    # [batch_size, hidden_size*2]

    
with tf.device('/gpu:0'):
    with tf.variable_scope('embedding') as vs:
        word_embedding = tf.get_variable(name="word_embedding", shape=wd_embedding.shape, 
                            initializer=tf.constant_initializer(wd_embedding), trainable=True)   # fine-tune
        char_embedding = tf.get_variable(name="char_embedding", shape=ch_embedding.shape, 
                            initializer=tf.constant_initializer(ch_embedding), trainable=True)   # fine-tune
        
with tf.name_scope('Inputs'):
    X1_inputs = tf.placeholder(tf.int64, [None, wd_sent_len], name='X1_input')  # wd_title
    X2_inputs = tf.placeholder(tf.int64, [None, wd_doc_len*wd_sent_len], name='X2_input')  # wd_content
    X3_inputs = tf.placeholder(tf.int64, [None, ch_sent_len], name='X3_input')  # ch_title
    X4_inputs = tf.placeholder(tf.int64, [None, ch_doc_len*ch_sent_len], name='X4_input')  # ch_content
    y_inputs = tf.placeholder(tf.float32, [None, n_class], name='y_input')    
    inputs1 = tf.nn.embedding_lookup(word_embedding, X1_inputs)
    inputs2 = tf.nn.embedding_lookup(word_embedding, X2_inputs)
    inputs3 = tf.nn.embedding_lookup(char_embedding, X3_inputs)
    inputs4 = tf.nn.embedding_lookup(char_embedding, X4_inputs)

with tf.variable_scope('wd-encoder'):    
    with tf.variable_scope('wd-title'):
        wd_output_title = bigru_reference(inputs1)                            # [batch_size, hidden_size*2]
    with tf.variable_scope('wd-content'):
        wd_output_content = han_reference(inputs2, sent_len=wd_sent_len)      # [batch_size, hidden_size*2]
    
with tf.variable_scope('ch-encoder'):
    with tf.variable_scope('ch-title'):
        ch_output_title = bigru_reference(inputs3)
    with tf.variable_scope('ch-content'):
        ch_output_content = han_reference(inputs4, sent_len=ch_sent_len)      # [batch_size, hidden_size*2]

    
with tf.variable_scope('fc-bn-layer'):
    output = tf.concat([wd_output_title, wd_output_content, ch_output_title, ch_output_content], axis=1)  # shape=[batch_size, n_filter_total*2]
    W_fc = weight_variable([hidden_size*8, fc_hidden_size], name='Weight_fc')
    tf.summary.histogram('W_fc', W_fc)
    h_fc = tf.matmul(output, W_fc, name='h_fc')
    beta_fc = bias_variable([fc_hidden_size], name="beta_fc")
    tf.summary.histogram('beta_fc', beta_fc)
    fc_bn, update_ema_fc = batchnorm(h_fc, tst, n_updates, beta_fc, convolutional=False)
    update_emas.append(update_ema_fc)
    fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
    fc_bn_drop = tf.nn.dropout(fc_bn_relu, keep_prob, name="fc_dropout")

with tf.variable_scope('out_layer'):
    W_out = weight_variable(shape=[fc_hidden_size, n_class], name='Weight_out')
    tf.summary.histogram('Weight_out', W_out)
    b_out = bias_variable([n_class], name='bias_out') 
    tf.summary.histogram('bias_out', b_out)
    y_pred = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  #每个类别的分数 scores
    
with tf.name_scope('cost'):
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_pred, labels=y_inputs))
    tf.summary.scalar('cost', cost)

#  ------------- 优化器设置 ---------------------
with tf.name_scope('AdamOptimizer1'):
    tvars1 = tf.trainable_variables()
    grads1, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars1), max_grad_norm)
    optimizer1 = tf.train.AdamOptimizer(learning_rate=lr)
    train_op1 = optimizer1.apply_gradients(zip(grads1, tvars1),
        global_step=tf.contrib.framework.get_or_create_global_step())
    
with tf.name_scope('AdamOptimizer2'):
    tvars2 = [tvar for tvar in tvars1 if 'embedding' not in tvar.name]
    grads2, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars2), max_grad_norm)
    optimizer2 = tf.train.AdamOptimizer(learning_rate=lr)
    train_op2 = optimizer2.apply_gradients(zip(grads2, tvars2),
        global_step=tf.contrib.framework.get_or_create_global_step())
update_op = tf.group(*update_emas)   # 更新 BN 参数


# summary
merged = tf.summary.merge_all() # summary
train_writer = tf.summary.FileWriter(summary_path + 'train', sess.graph)
test_writer = tf.summary.FileWriter(summary_path + 'test')
print 'Finished creating the bigru model.'

Building model ...
Finished creating the bigru model.


## 导入数据

In [3]:
sys.path.append('..')
from data_helpers import BatchGenerator
from data_helpers import to_categorical
from evaluator import score_eval


wd_train_path = '/home/peng/huangyongye/wd-data/seg_train/'
wd_valid_path = '/home/peng/huangyongye/wd-data/seg_valid/'
ch_train_path = '/home/peng/huangyongye/ch-data/seg_train/'
ch_valid_path = '/home/peng/huangyongye/ch-data/seg_valid/'


tr_batches = os.listdir(wd_train_path)   # batch 文件名列表
va_batches = os.listdir(wd_valid_path)
n_tr_batches = len(tr_batches)
n_va_batches = len(va_batches)

# 测试用
# n_tr_batches = 500  
# n_va_batches = 50
print('n_tr_batch=%d' % n_tr_batches)
print('n_va_batches=%d' % n_va_batches)


def get_batch(batch_id, mode='train'):
    """get a batch from data_path.
    model: 'train', 'valid' 三种方式。
    """
    if mode == 'train':
        wd_path = wd_train_path
        ch_path = ch_train_path 
    if mode == 'valid':
        wd_path = wd_valid_path
        ch_path = ch_valid_path
    wd_batch = np.load(wd_path + str(batch_id) + '.npz')
    wd_X_batch = wd_batch['X']
    X1_batch = wd_X_batch[:, :n_step1]
    X2_batch = wd_X_batch[:, n_step1:]
    ch_batch = np.load(ch_path + str(batch_id) + '.npz')
    ch_X_batch = ch_batch['X']
    X3_batch = ch_X_batch[:, :n_step3]
    X4_batch = ch_X_batch[:, n_step3:]
    y_batch = ch_batch['y']
    return [X1_batch, X2_batch, X3_batch, X4_batch, y_batch]


def valid_epoch():
    """Test on the valid data."""
    _costs = 0.0
    predict_labels_list = list()  # 所有的预测结果
    marked_labels_list = list()   # 真实标签
    for i in xrange(n_va_batches):
        [X1_batch, X2_batch, X3_batch, X4_batch, y_batch] = get_batch(i, mode='valid')
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        fetches = [merged, cost, y_pred]  
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch, y_inputs:y_batch, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        summary, _cost, predict_labels = sess.run(fetches, feed_dict)
        _costs += _cost
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    mean_cost = _costs / n_va_batches
    return mean_cost, precision, recall, f1

print('Every thing prepared!')

n_tr_batch=22656
n_va_batches=782
Every thing prepared!


##  模型训练

之前的训练速度 9334.15 s/epoch

In [4]:
_lr = 1.5e-3
decay = 0.85 
# 正式
max_epoch = 3            # 超多 max_epoch 以后开始降低 lr
max_max_epoch = 9       # 最多迭代的次数
valid_step = 8000        # 每 valid_step 就进行一次 valid 运算
decay_step = 11000

# 测试
# max_epoch = 1            # 超多 max_epoch 以后开始降低 lr
# max_max_epoch = 3       # 最多迭代的次数
# decay_step = 500
# valid_step = 250

print('tr_batch_num=%d' % n_tr_batches)
saver = tf.train.Saver(max_to_keep=3)           # 最多保存的模型数量
sess.run(tf.global_variables_initializer())
last_f1 = 0.40
model_num = 0
global_step = 0
print('OK!')

tr_batch_num=22656
OK!


In [5]:
train_op = train_op2
time0 = time.time()

for epoch in xrange(max_max_epoch):
    batch_indexs = np.random.permutation(n_tr_batches)  # shuffle the training data
    if epoch == max_epoch:
        train_op = train_op1
        if model_num == 0:
            model_num += 1
            save_path = saver.save(sess, model_path, global_step=model_num)
            print('the save path is ', save_path)
        print('Begin updating embedding.')
    print 'EPOCH %d, lr= %g' % (epoch+1, _lr)    
    for batch in xrange(n_tr_batches): 
        global_step += 1
        if (global_step+1) % decay_step == 0:
            _lr = _lr * decay
        if (global_step+1) % valid_step == 0:    # 进行 valid 计算
            valid_cost, precision, recall, f1 = valid_epoch()
            print('Global_step=%d: valid cost=%g; p=%g, r=%g, f1=%g, time=%gs' % (
                    global_step+1, valid_cost, precision, recall, f1, time.time()-time0))
            time0 = time.time()
            if (f1 > last_f1):
                last_f1 = f1
                model_num += 1
                save_path = saver.save(sess, model_path, global_step=model_num)
                print('The save path is ', save_path)
                
        batch_id = batch_indexs[batch]
        [X1_batch, X2_batch, X3_batch, X4_batch, y_batch] = get_batch(batch_id, mode='train')
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        fetches = [merged, cost, train_op, update_op]
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch, y_inputs:y_batch, 
                     batch_size:_batch_size, lr:_lr, keep_prob:0.5, tst:False, n_updates:global_step}
        summary, _cost, _, _ = sess.run(fetches, feed_dict) # the cost is the mean cost of one batch
        if global_step % 500:
            train_writer.add_summary(summary, global_step)
            batch_id = np.random.randint(0, n_va_batches)   # 随机选一个验证batch
            [X1_batch, X2_batch, X3_batch, X4_batch, y_batch] = get_batch(batch_id, mode='valid')
            y_batch = to_categorical(y_batch)
            _batch_size = len(y_batch)
            feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch, y_inputs:y_batch, 
                         batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
            fetches = [merged, cost]
            summary, _cost = sess.run(fetches, feed_dict)
            test_writer.add_summary(summary, global_step)

valid_cost, precision, recall, f1 = valid_epoch()  # # 每个 epoch 进行一次验证 valid
print('Global_step=%d;  valid cost=%g; p=%g, r=%g, f1=%g; speed=%g s/epoch' % (
    global_step+1, valid_cost, precision, recall, f1, time.time()-time0) )
if (f1 > last_f1):
    model_num += 1
    save_path = saver.save(sess, model_path, global_step=model_num)
    print('the save path is ', save_path) 

EPOCH 1, lr= 0.0015
Global_step=8000: valid cost=0.00464288; p=1.19449, r=0.491513, f1=0.348225, time=4256.25s
Global_step=16000: valid cost=0.00432813; p=1.26633, r=0.51939, f1=0.368321, time=4257.86s
EPOCH 2, lr= 0.00108375
Global_step=24000: valid cost=0.00422078; p=1.29733, r=0.530832, f1=0.376698, time=4256.84s
Global_step=32000: valid cost=0.00420759; p=1.31526, r=0.537754, f1=0.381695, time=4271.25s
Global_step=40000: valid cost=0.00415157; p=1.32378, r=0.540952, f1=0.384023, time=4279.22s
EPOCH 3, lr= 0.000783009
Global_step=48000: valid cost=0.00410971; p=1.33327, r=0.545172, f1=0.386949, time=4273.63s
Global_step=56000: valid cost=0.00407327; p=1.34816, r=0.550414, f1=0.390844, time=4270.97s
Global_step=64000: valid cost=0.00405052; p=1.35365, r=0.552047, f1=0.392129, time=4267.94s
('the save path is ', '../ckpt/c2-1-bigru-256/model.ckpt-1')
Begin updating embedding.
EPOCH 4, lr= 0.000565724
Global_step=72000: valid cost=0.0040885; p=1.35742, r=0.553244, f1=0.39305, time=4507

KeyboardInterrupt: 

- Bi-LSTM 模型<br/>
**batch_size=256，迭代12个epoch，基本收敛。结果： 验证集 f1=0.38618； 提交结果 0.3873186**
- Bi-GRU 模型<br/>
**batch_size=256，迭代15个epoch。大概在 13 个 epoch 就收敛了。结果： 验证集 f1=0.390534； 提交结果 0.39198**
- attention-Bi-GRU 模型<br/>
**batch_size=256，迭代18个epoch。在16个epoch收敛。结果：验证集 f1=f1=0.391734 ；提交结果 0.39310**


- textcnn-256 lr=1e-3, decay=0.65, dropout=0.5, 迭代6次基本收敛， f1=0.388
- textcnn-256-bn lr=1.5E-3，decay=0.65, dropout=0.5, 6次基本收敛，8次好一点， f1=0.389
- textcnn-fc-drop 迭代 4.5 个epoch 收敛。

## 本地测试
使用 seed13 的前 10万条数据.

In [8]:
# 导入保存好的模型
saver = tf.train.Saver()
best_model_path = model_path + '-' + str(7)  # 导入最优模型
saver.restore(sess, best_model_path)
print('Finished loading model.')

from tqdm import tqdm
local_scores_path = '../local_scores/' + model_name + '.npy'

# 导入测试数据
def local_predict(scores_path=local_scores_path):
    """预测  valid 结果，并保存预测概率 到  scores.csv 文件中。"""
    print('local predicting ...')
    time0 = time.time()
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    marked_labels_list = list()   # 真实标签
    for i in tqdm(xrange(n_va_batches)):
        [X1_batch, X2_batch, X3_batch, X4_batch, y_batch] = get_batch(i, mode='valid')
        marked_labels_list.extend(y_batch)
        y_batch = to_categorical(y_batch)
        _batch_size = len(y_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch,
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
    precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
    print('local valid p=%g, r=%g, f1=%g; speed=%g s/epoch' % ( precision, recall, f1, time.time()-time0) )
    np.save(local_scores_path, predict_scores)
    print('Writed the scores into %s, time %g s' % (local_scores_path, time.time()-time0))
    
local_predict()

INFO:tensorflow:Restoring parameters from ../ckpt/c2-1-bigru-256/model.ckpt-7


  0%|          | 1/782 [00:00<02:14,  5.80it/s]

Finished loading model.
local predicting ...


100%|██████████| 782/782 [02:13<00:00,  6.27it/s]


local valid p=1.40017, r=0.57006, f1=0.405121; speed=134.006 s/epoch
Writed the scores into ../local_scores/c2-1-bigru-256.npy, time 138.159 s


## 对测试数据进行预测

In [9]:
def get_test_batch(batch_id):
    """get a batch from data_path.
    model: 'train', 'valid' 三种方式。
    """
    wd_path = '/home/peng/huangyongye/wd-data/seg_test/'
    ch_path = '/home/peng/huangyongye/ch-data/seg_test/'
    wd_X_batch = np.load(wd_path + str(batch_id) + '.npy')
    X1_batch = wd_X_batch[:, :n_step1]
    X2_batch = wd_X_batch[:, n_step1:]
    ch_X_batch = np.load(ch_path + str(batch_id) + '.npy')
    X3_batch = ch_X_batch[:, :n_step3]
    X4_batch = ch_X_batch[:, n_step3:]
    return [X1_batch, X2_batch, X3_batch, X4_batch]


In [23]:
topics_ids = np.load('../data/topics_ids.npz')
sr_id2topic = pd.Series(topics_ids['topics'], index=topics_ids['ids'])
sr_topic2id = pd.Series(topics_ids['ids'], index=topics_ids['topics'])

In [24]:
# 导入保存好的模型
# saver = tf.train.Saver()
# best_model_path = model_path + '-' + str(8)  # 导入最优模型
# saver.restore(sess, best_model_path)
# print('Finished loading model.')

# 导入测试数据
def predict():
    """预测测试集结果，并保存到  result.csv 文件中。"""
    n_te_batches = len(os.listdir('/home/peng/huangyongye/wd-data/seg_test/'))
    fetches = [y_pred]   
    predict_labels_list = list()  # 所有的预测结果
    predict_scores = list()
    for i in xrange(n_te_batches):
        [X1_batch, X2_batch, X3_batch, X4_batch] = get_test_batch(i)
        _batch_size = len(X1_batch)
        feed_dict = {X1_inputs:X1_batch, X2_inputs:X2_batch, X3_inputs:X3_batch, X4_inputs:X4_batch, 
                     batch_size:_batch_size, keep_prob:1.0, tst:True, n_updates:global_step}
        predict_labels = sess.run(fetches, feed_dict)[0]
        predict_scores.append(predict_labels)
        predict_labels = map(lambda label: label.argsort()[-1:-6:-1], predict_labels) # 取最大的5个下标
        predict_labels_list.extend(predict_labels)
    predict_scores = np.asarray(predict_scores)
    return predict_labels_list, np.vstack(predict_scores)

def write_result(predict_labels_list, result_path):
    """把结果写到 result.csv 中"""
    eval_question = np.load('../data/eval_question.npy')
    pred_labels = np.asarray(predict_labels_list).reshape([-1])
    pred_topics = sr_id2topic[pred_labels].values.reshape([-1, 5])   # 转为 topic
    df_result = pd.DataFrame({'question':eval_question, 'tid0': pred_topics[:,0], 'tid1':pred_topics[:, 1],
                         'tid2': pred_topics[:,2], 'tid3':pred_topics[:,3],'tid4': pred_topics[:,4]})
    df_result.to_csv(result_path, index=False, header=False)
    print('Finished writing the result')
    return df_result

In [25]:
# %time predict_labels_list,predict_scores = predict()
df_result = write_result(predict_labels_list, result_path=result_path) 
print('len(df_result)=',len(df_result))  # 结果应该为 217360
print('Saving the predict_scores into %s' % scores_path)
print('predict_scores.shape=',predict_scores.shape)
np.save(scores_path, predict_scores)
print('Finished saving the result!')

Finished writing the result
('len(df_result)=', 217360)
Saving the predict_scores into ../scores/c2-1-bigru-256.npy
('predict_scores.shape=', (217360, 1999))
Finished saving the result!


In [26]:
df_result.head(15)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,-7506384235581390893,-6839713564940654454,-6306904715218704629,2919247920214845195
1,6649324930261961840,-240041917918953337,3418451812342379591,2858911571784840089,-7483543763655495143,9149698301250936831
2,-4251899610700378615,2919247920214845195,-7358589937244777363,-7270992690764838239,2816249700493135244,-5265476641576484497
3,6213817087034420233,5804619920623030604,-8655945395761165989,-4966205278807386328,7476760589625268543,244937959911721367
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,-985918886318729368,1870872991887862017
5,-2893445476547593888,-6206436693745657677,2131451573312950491,-2696736445927423374,-8274522839089381384,-3900318848301728120
6,2614833994648160978,4482402820945758152,-3856154743789177934,-6748914495015758455,-3904210248050890128,1553849747662134917
7,1572988006266661060,-839691564858654120,3738968195649774859,-8932546057542867495,-2627298052801704596,1160326435131345730
8,-3736249911643942320,-7653703019053330516,8690965822342756180,738845194850773558,1127459907694805235,5502528845814007324
9,-976507019126932319,3195914392210930723,3804601920633030746,6940355838132160535,701949295768128821,6718676536613592056


In [9]:
# 参考结果
df_result.head(5)

,question,tid0,tid1,tid2,tid3,tid4
0,6215603645409872328,4610596224687453206,-6839713564940654454,-6306904715218704629,2919247920214845195,-8091907016971478715
1,6649324930261961840,3418451812342379591,2858911571784840089,2382911985227044227,-240041917918953337,3383016985780045156
2,-4251899610700378615,2919247920214845195,-7358589937244777363,2816249700493135244,-5265476641576484497,-3315241959305847628
3,6213817087034420233,-8655945395761165989,5804619920623030604,-4966205278807386328,7476760589625268543,-2523521411748733187
4,-8930652370334418373,3972493657017129406,-8963554618409314978,-1115593437686158905,1870872991887862017,6018641953300645757


## 在全部预测正确的情况下，理论值为：f1=0.713933
precision=2.50273, recall=0.998873, f1=0.713933

In [26]:
# 假设全部正确，f1 值最高能到多少
def padding_label(labels):
    """把所有的label补齐到长度为 5"""
    label_len = len(labels)
    if label_len >= 5:
        return labels[:5]
    return np.hstack([labels, np.zeros(5-label_len, dtype=int) - 1])
    

marked_labels_list = data_valid.y.tolist() # 所有的标注结果
predict_labels_list = map(padding_label, marked_labels_list)
predict_label_and_marked_label_list = zip(predict_labels_list, marked_labels_list)
precision, recall, f1 = score_eval(predict_label_and_marked_label_list)
print '在全部预测正确的情况下，理论值为：'
print 'precision=%g, recall=%g, f1=%g' % (precision, recall, f1)

在全部预测正确的情况下，理论值为：
precision=2.50273, recall=0.998873, f1=0.713933
